In [ ]:
#creating tables using SQLlite
import sqlite3
import json
def update_database():

    
    
    # Connect to SQLite database (or create it)
    conn = sqlite3.connect('inventory_management.db')
    cursor = conn.cursor()
    
    # Create Users table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            mobile TEXT,
            country TEXT
        )
    ''')
    
    # Create Items table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Items (
            product_id TEXT PRIMARY KEY,
            name TEXT,
            price REAL,
            engine TEXT,
            quantity INTEGER
        )
    ''')
    
    # Create Sales table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Sales (
            transaction_id TEXT PRIMARY KEY,
            user_id INTEGER,
            product_id TEXT,
            quantity INTEGER,
            billing_amount REAL,
            timestamp TEXT,
            FOREIGN KEY (user_id) REFERENCES Users(user_id),
            FOREIGN KEY (product_id) REFERENCES Items(product_id)
        )
    ''')
    
    conn.commit()
    
    
    #extracting data from the created JSON files
    import json
    
    with open('C:/Users/PlethoraX/Desktop/Records.json', 'r') as f:
        records = json.load(f)
    
    # Load sales.json
    with open('C:/Users/PlethoraX/Desktop/sales.json', 'r') as f:
        sales_data = json.load(f)
    
    # Insert data into the Items table
    conn = sqlite3.connect('inventory_management.db')
    cursor = conn.cursor()
    
    # Perform database operations
    
    
    for product_id, details in records.items():
        cursor.execute('''
            INSERT OR REPLACE INTO Items (product_id, name, price, engine, quantity)
            VALUES (?, ?, ?, ?, ?)
        ''', (product_id, details['Name'], details['Price'], details['Engine'], int(details['Quantity'])))
        
    conn.commit()
    # Extract and transform sales data, and insert into Users and Sales tables
    
    conn = sqlite3.connect('inventory_management.db')
    cursor = conn.cursor()
    
    for transaction_id, sale in sales_data.items():
        # Insert user details into Users table (if not already present)
        cursor.execute('''
            INSERT OR IGNORE INTO Users (name, mobile, country)
            VALUES (?, ?, ?)
        ''', (sale['Name'], sale['Mobile'], sale['Country']))
        
        # Get the user_id for the inserted/ignored user
        cursor.execute('SELECT user_id FROM Users WHERE mobile = ?', (sale['Mobile'],))
        user_id = cursor.fetchone()[0]
    
        # Insert sales record into Sales table with the provided transaction_id
        cursor.execute('SELECT COUNT(*) FROM Sales WHERE transaction_id = ?', (transaction_id,))
        if cursor.fetchone()[0] == 0:  # If count is 0, the transaction_id does not exist
            # Insert sales record into Sales table with the provided transaction_id
            cursor.execute('''
                INSERT INTO Sales (transaction_id, user_id, product_id, quantity, billing_amount, timestamp)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (transaction_id, user_id, sale['Product_ID'], sale['Quantity'], float(sale['Billing_Amount']), sale['Timestamp']))
        else:
            pass
    conn.commit()
    conn.close()


In [3]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('inventory_management.db')
query = '''
    SELECT Users.user_id, Users.name, Users.mobile, Users.country, 
           Sales.transaction_id,Sales.product_id, Sales.quantity, Sales.billing_amount, Sales.timestamp
    FROM Users
    JOIN Sales ON Users.user_id = Sales.user_id
'''
df = pd.read_sql_query(query, conn)

In [4]:
df

,user_id,name,mobile,country,transaction_id,product_id,quantity,billing_amount,timestamp
0,1,Dibbyo,9876543234,France,20738,1004,40,118000000.0,Thu Sep 5 15:45:52 2024
1,2,Daniel Radcliffe,98765432654,USA,21722,1003,30,40500000.0,Thu Sep 5 15:49:40 2024
2,5,Housty James,34567890876,USA,45298,1004,30,88500000.0,Fri Sep 6 17:57:48 2024
3,6,Janet Bethlam,234567890,USA,94162,1001,50,12500000.0,Fri Sep 6 17:58:18 2024
4,15,Damos Janes,45678908,Africa,19617,1005,20,85000000.0,Fri Sep 6 18:00:40 2024
5,16,Tomaz perox,987654567890,France,20071,1004,20,59000000.0,Fri Sep 6 18:01:06 2024
6,41,Jack Machismo,234567876543,UK,23757,1005,40,170000000.0,Fri Sep 6 19:23:37 2024
7,42,Davis James,345678987654,UK,83518,1002,20,11000000.0,Fri Sep 6 19:31:44 2024
8,43,Dack mory,234565432,IRELAND,34643,1003,2,2700000.0,Fri Sep 6 20:08:17 2024
9,44,DACKER MORTY,345432344,FRANCE,86927,1003,20,27000000.0,Fri Sep 6 20:11:27 2024


# #checking created tables
# conn = sqlite3.connect('inventory.db')
# cursor = conn.cursor()
    
# # List tables
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = cursor.fetchall()
# print("Tables:", tables)
    
# conn.close()